In [47]:
# the following code adds a new meaning for python '<=' on TypedExprs at type e.  This is
# intended to be treated as a part-of operator, though there are no semantics per se.
class PartOf(meta.BinaryOpExpr):
    """Part Of"""
    def __init__(self, arg1, arg2):
        arg1 = self.ensure_typed_expr(arg1, meta.type_e)
        arg2 = self.ensure_typed_expr(arg2, meta.type_e)
        super().__init__(meta.type_t, "<=", arg1, arg2, op_name_uni = "<=", op_name_latex = "\\sqsubseteq", tcheck_args = False)

# by default, '<=" is just mapped to numeric.  This overrides that and allows for 
# two possibilities.  Note that this won't play all that nicely with type variables.
def leq_factory(arg1, arg2):
    # see meta.eq_factory, which does something similar for "<=>".
    arg1 = meta.TypedExpr.ensure_typed_expr(arg1)
    arg2 = meta.TypedExpr.ensure_typed_expr(arg2)
    ts = meta.get_type_system()
    if ts.eq_check(arg1.type, types.type_n):
        return meta.BinaryLeqExpr(arg1, arg2)
    elif ts.eq_check(arg1.type, types.type_e):
        return PartOf(arg1, arg2)
    else:
        raise parsing.ParseError("Unknown type '%s' for <= operator." % arg1.type)

# replace default, which would just be the constructor for BinaryLeqExpr
meta.binary_symbols_to_op_exprs["<="] = leq_factory

In [48]:
x = PartOf(te("x_e"), te("y_e"))
x

(x_e <= y_e)

In [49]:
%te x_e <= y_e

(x_e <= y_e)

In [52]:
# Now do something similar for '+'.
class MSum(meta.BinaryOpExpr):
    """Mereological sum"""
    def __init__(self, arg1, arg2):
        # use the default type-checking, because the args are the same type as the output
        super().__init__(meta.type_e, "+", arg1, arg2, op_name_uni = "+", op_name_latex = "\\oplus")

def sum_factory(arg1, arg2):
    arg1 = meta.TypedExpr.ensure_typed_expr(arg1)
    arg2 = meta.TypedExpr.ensure_typed_expr(arg2)
    ts = meta.get_type_system()
    if ts.eq_check(arg1.type, types.type_n):
        return meta.BinaryPlusExpr(arg1, arg2)
    elif ts.eq_check(arg1.type, types.type_e):
        return MSum(arg1, arg2)
    else:
        raise parsing.ParseError("Unknown type '%s' for + operator." % arg1.type)

# replace default, which would just be the constructor for BinaryPlusExpr
meta.binary_symbols_to_op_exprs["+"] = sum_factory

In [58]:
%te (x_e + y_e) <= z_e # demonstrate with meta-language operators

((x_e + y_e) <= z_e)

In [59]:
te("x_e") + te("y_e") <= te("z_e") # demonstrate with python operators

((x_e + y_e) <= z_e)